# This is to get the grid level filtered tif.

In [1]:
import rasterio
from rasterio.plot import show
from rasterio.mask import mask
import pandas as pd
import numpy as np
import geopandas as gpd
import glob
from shapely.geometry import Point
from shapely.geometry import box
import os
import csv

1. 拿到一个国家的shp 然后拿到 国家.geometry.bounds
2. 对每一个 tif文件,
    用每一个 国家 mask一下: #masked, out_transform = mask(src, [bbox], crop=True)
        之后生成一个 masked np array
        out_transform 是变换矩阵 #lon, lat = transform * (indexx, indexy)
        然后 loop 这里面的point(indexx, indexy):
            拿值 拿lonlat 拿名字 存到csv里

In [2]:
# Read the shapefile
gdf = gpd.read_file('./country_poly_data/WB_countries_Admin0_10m.shp')

### Get names from ISO of old world countries

In [3]:
# Read the datasets
country_old_world = pd.read_csv('./country_old_world.csv')
len_old = len(country_old_world['ISO'])
print('Number of countries in old world: ', len_old)
country_name_codes = pd.read_csv('./country_name_codes.csv')

# Perform the inner join on the 'ISO' column
merged_data = country_old_world.merge(country_name_codes, on='ISO', suffixes=('_old', '_new'))
len_new = len(merged_data['ISO'])
print('Number of countries after merging: ', len_new)
# # Save the result to a new CSV file
# merged_data.to_csv('merged_data.csv', index=False)


if len_old != len_new:
        # Merge the datasets using an outer join
    merged_df = country_old_world.merge(country_name_codes, on='ISO', how='outer', indicator=True)

    # Filter the rows that are only in the 'old_world' dataset
    missing_iso = merged_df[merged_df['_merge'] == 'left_only']

    # Print the ISO codes that are in 'old_world' but not in 'country_name'
    print('The following columns are missing:', missing_iso['ISO'].tolist())

Number of countries in old world:  143
Number of countries after merging:  142
The following columns are missing: ['COD']


### Code to get the country boundaries from shp file

In [4]:
def get_country_boundary(gdf, country_name):
    # Filter the GeoDataFrame for the specific country
    country_gdf = gdf[gdf['NAME_EN'].str.lower() == country_name.lower()]
    
    if country_gdf.empty:
        print("Country not found: ", country_name)
        return None

    # Return the GeoSeries of the country boundary geometry
    return country_gdf.geometry

# Initialize an empty dictionary to store country boundaries
# Store all the boundaries required.
country_boundaries = {}

# Loop through the merged_data DataFrame
# Get all needed countries' boundary boxed into a dict.
for index, row in merged_data.iterrows():
    country_name = row['Country_name']
    country_geometry = get_country_boundary(gdf, country_name)

    if country_geometry is not None:
        minx, miny, maxx, maxy = country_geometry.total_bounds
        bbox = box(minx, miny, maxx, maxy)
        
        country_boundaries[country_name] = bbox


        

Country not found:  Cote d'Ivoire
Country not found:  Ethiopia (excludes Eritrea)
Country not found:  Faeroe Islands
Country not found:  Gambia, The
Country not found:  Kyrgyz Republic
Country not found:  Lao PDR
Country not found:  Macao
Country not found:  Macedonia, FYR
Country not found:  Sao Tome and Principe
Country not found:  Slovak Republic
Country not found:  Syrian Arab Republic
Country not found:  Yemen, Rep.
Country not found:  Yugoslavia, FR (Serbia/Montene


1. 拿到一个国家的shp 然后拿到 国家.geometry.total bounds
2. 对每一个 tif文件,
    用每一个 国家 mask一下: #masked_img, out_transform = mask(src, [boundary_box], crop=True)
        之后生成一个 masked np array
        out_transform 是变换矩阵 #lon, lat = transform * (indexx, indexy)
        然后 loop 这里面的point(indexx, indexy):
            拿值 拿lonlat 拿名字 存到csv里

# Try one file

In [16]:

file_name = 'F101992.v4b_web.stable_lights'
tif_file ='./data/F101992.v4b_web.stable_lights.avg_vis.tif'
# Load the tif data using the rasterio module
with rasterio.open(tif_file) as src:
    # Make a list to save the latitude, longitude, value, and country_name 
    csv_rows = []
    
    #  loop through the dict of {country_name : boundary_box}
    for country_name, boundary_box in country_boundaries.items():
        # filter the tif file.
        # 'masked_img' is the filtered tif data. shape = (1, height, length)
        # 'out_transform' is transform matrix, which uses the index of pixel to get its real lat/lon.
        # Get the pixel level first then sample them to be the 5-5 grid level (using the result = matrix[::5, ::5] )
        masked_img, out_transform = mask(src, [boundary_box], crop=True)
        masked_values = masked_img[0]

        # TODO: after using the np to replace the for loop, get the value with two matrixes.
        # Get the cols and rows matrix for each pixel
        cols, rows =  np.meshgrid(np.arange(masked_values.shape[1]),np.arange(masked_values.shape[0]))
        # Turn to real lon lat for each index. Get location first then do the sampling.
        # lon.shape = 1092, 1730 = lat.shape
        lon, lat = rasterio.transform.xy(out_transform, rows, cols)
        lon = np.array(lon)
        lat = np.array(lat)

        # Sample the values and lat lon for every 5th point to be a 5-5 grid.
        # The following 3 matrixes are linked together. 对应位置一一对照
        masked_values_sampled = masked_values[::5, ::5] # e.g. masked_values_sampled[x,y] = 10. This means masked_values_sampled[x,y]'s real value is 10.
        lon_sampled = lon[::5, ::5] # e.g. lon_sampled[x,y] = 3. This means lon_sampled[x,y]'s real lon is 3.
        lat_sampled = lat[::5, ::5] # e.g. lat_sampled[x,y] = 4. This means lat_sampled[x,y]'s real lat is 4. 

        

        # Loop each item
        for i in range(masked_values_sampled.shape[0]):
            for j in range(masked_values_sampled.shape[1]):
                point_lon = lon_sampled[i,j]
                point_lat = lat_sampled[i,j]
                point_value = masked_values_sampled[i,j]

                
                csv_rows.append((point_lat, point_lon, point_value, country_name))


    # Save the data of csv_rows into a csv file        
    # Write the list to a CSV file with the same name as the tif file
    with open(f'./extracted_data/{file_name}.csv', 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['latitude', 'longitude', 'value', 'country_name'])
        csv_writer.writerows(csv_rows)
    print('Finished for: ', tif_file)        

            


Finished for:  ./data/F101992.v4b_web.stable_lights.avg_vis.tif


## Loop all files

In [17]:
# Loop all the tif files in the folder 
for tif_file in glob.glob('./data/*.tif'):
    # Get the file name without the extension.
    # Use the name to save the filtered data later.
    file_name = os.path.splitext(os.path.basename(tif_file))[0]

    # Load the tif data using the rasterio module
    with rasterio.open(tif_file) as src:
        # Make a list to save the latitude, longitude, value, and country_name 
        csv_rows = []
        
        #  loop through the dict of {country_name : boundary_box}
        for country_name, boundary_box in country_boundaries.items():
            # filter the tif file.
            # 'masked_img' is the filtered tif data. shape = (1, height, length)
            # 'out_transform' is transform matrix, which uses the index of pixel to get its real lat/lon.
            # Get the pixel level first then sample them to be the 5-5 grid level (using the result = matrix[::5, ::5] )
            masked_img, out_transform = mask(src, [boundary_box], crop=True)
            masked_values = masked_img[0]

            # TODO: after using the np to replace the for loop, get the value with two matrixes.
            # Get the cols and rows matrix for each pixel
            cols, rows =  np.meshgrid(np.arange(masked_values.shape[1]),np.arange(masked_values.shape[0]))
            # Turn to real lon lat for each index. Get location first then do the sampling.
            # lon.shape = 1092, 1730 = lat.shape
            lon, lat = rasterio.transform.xy(out_transform, rows, cols)
            lon = np.array(lon)
            lat = np.array(lat)

            # Sample the values and lat lon for every 5th point to be a 5-5 grid.
            # The following 3 matrixes are linked together. 对应位置一一对照
            masked_values_sampled = masked_values[::5, ::5] # e.g. masked_values_sampled[x,y] = 10. This means masked_values_sampled[x,y]'s real value is 10.
            lon_sampled = lon[::5, ::5] # e.g. lon_sampled[x,y] = 3. This means lon_sampled[x,y]'s real lon is 3.
            lat_sampled = lat[::5, ::5] # e.g. lat_sampled[x,y] = 4. This means lat_sampled[x,y]'s real lat is 4. 

            

            # Loop each item
            for i in range(masked_values_sampled.shape[0]):
                for j in range(masked_values_sampled.shape[1]):
                    point_lon = lon_sampled[i,j]
                    point_lat = lat_sampled[i,j]
                    point_value = masked_values_sampled[i,j]

                    csv_rows.append((point_lat, point_lon, point_value, country_name))


        # Save the data of csv_rows into a csv file        
        # Write the list to a CSV file with the same name as the tif file
        with open(f'./extracted_data/{file_name}.csv', 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(['latitude', 'longitude', 'value', 'country_name'])
            csv_writer.writerows(csv_rows)
        print('Finished for: ', tif_file)     

            




Finished for:  ./data/F152000.v4b_web.stable_lights.avg_vis.tif
Finished for:  ./data/F142000.v4b_web.stable_lights.avg_vis.tif
Finished for:  ./data/F142002.v4b_web.stable_lights.avg_vis.tif
Finished for:  ./data/F101992.v4b_web.stable_lights.avg_vis.tif
Finished for:  ./data/F152002.v4b_web.stable_lights.avg_vis.tif
Finished for:  ./data/F142001.v4b_web.stable_lights.avg_vis.tif
Finished for:  ./data/F152001.v4b_web.stable_lights.avg_vis.tif
Finished for:  ./data/F152003.v4b_web.stable_lights.avg_vis.tif
Finished for:  ./data/F101993.v4b_web.stable_lights.avg_vis.tif
Finished for:  ./data/F142003.v4b_web.stable_lights.avg_vis.tif
Finished for:  ./data/F162008.v4b_web.stable_lights.avg_vis.tif
Finished for:  ./data/F121998.v4b_web.stable_lights.avg_vis.tif
Finished for:  ./data/F152005.v4b_web.stable_lights.avg_vis.tif
Finished for:  ./data/F162005.v4b_web.stable_lights.avg_vis.tif
Finished for:  ./data/F141998.v4b_web.stable_lights.avg_vis.tif
Finished for:  ./data/F121995.v4b_web.st